In [1]:
import pandas as pd
from mesmerize.plotting.utils import get_colormap

/home/kushal/python-venvs/mesmerize/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df = pd.read_csv('./Behavior2.csv')

**Behavior is marked for each timepoint, we need to convert this into a "chunked" representation where each row has a ``start``, and ``end`` frame number for each behavior**

In [3]:
df

,Time,Behavior
0,1.63,L
1,1.65,L
2,1.68,L
3,1.70,L
4,1.73,L
...,...,...
11300,284.13,S
11301,284.15,S
11302,284.18,S
11303,284.20,S


**The framerate of the movie is known as 10fps, so we change the time units from seconds to frames**

In [4]:
df['Time'] = df['Time'].apply(lambda x: int(x*10))

**Some cells in the CSV were blank, let's just label them as "nothing". Do not label them as ``None``!**

In [5]:
df['Behavior'].unique()

array(['L', 'S', 'O', 'Z', 'R', 'A', 'T', nan], dtype=object)

In [6]:
df['Behavior'] = df['Behavior'].fillna('nothing')
df['Behavior'].unique()

array(['L', 'S', 'O', 'Z', 'R', 'A', 'T', 'nothing'], dtype=object)

**Create a colormap that will be used to illustrate the behavioral periods in the Viewer**

**The output MUST be set to ``pyqt``**

In [7]:
behavior_name_cmap = get_colormap(df['Behavior'].unique(), 'tab10', output='pyqt', alpha=0.6)
behavior_name_cmap

OrderedDict([('L', <PyQt5.QtGui.QColor at 0x7fd0018de438>),
             ('S', <PyQt5.QtGui.QColor at 0x7fd0018de4a8>),
             ('O', <PyQt5.QtGui.QColor at 0x7fd0018de518>),
             ('Z', <PyQt5.QtGui.QColor at 0x7fd0018de588>),
             ('R', <PyQt5.QtGui.QColor at 0x7fd0018de5f8>),
             ('A', <PyQt5.QtGui.QColor at 0x7fd0018de668>),
             ('T', <PyQt5.QtGui.QColor at 0x7fd0018de6d8>),
             ('nothing', <PyQt5.QtGui.QColor at 0x7fd0018de748>)])

**Create an empty dataframe with the required columns**

In [8]:
stim_mapping_df = pd.DataFrame(columns=['start', 'end', 'name', 'color'])

**Fill in the dataframe**

In [9]:
for i, r in df.iterrows():
    if i == 0:  # the inital behavioral period
        start = r['Time']  # the first frame
        behavior = r['Behavior']  # the first behavior name
        continue
    
    # when it encounters that the behavioral period has changed, output the previous period
    if behavior != r['Behavior']:
        behavioral_period = pd.Series(
            {
                'start': int(start),
                'end': int(df.iloc[i-1]['Time']),
                'name': behavior,
                'color': behavior_name_cmap[behavior]
            }
        )
        # append the previous behavioral period
        stim_mapping_df = stim_mapping_df.append(behavioral_period, ignore_index=True)
        
        # get the start time and name of the new behavioral period
        start = r['Time']
        behavior = r['Behavior']

**View the dataframe which can be fed to the Stimulus Mapping Module of Mesmerize**

In [10]:
stim_mapping_df

,start,end,name,color
0,16,49,L,<PyQt5.QtGui.QColor object at 0x7fd0018de438>
1,49,76,S,<PyQt5.QtGui.QColor object at 0x7fd0018de4a8>
2,76,122,L,<PyQt5.QtGui.QColor object at 0x7fd0018de438>
3,122,179,O,<PyQt5.QtGui.QColor object at 0x7fd0018de518>
4,179,199,S,<PyQt5.QtGui.QColor object at 0x7fd0018de4a8>
...,...,...,...,...
149,2725,2730,A,<PyQt5.QtGui.QColor object at 0x7fd0018de668>
150,2730,2740,T,<PyQt5.QtGui.QColor object at 0x7fd0018de6d8>
151,2740,2761,S,<PyQt5.QtGui.QColor object at 0x7fd0018de4a8>
152,2761,2794,L,<PyQt5.QtGui.QColor object at 0x7fd0018de438>
